<h1> Algorithmen in der Algebra, Woche 8 </h1>
<h2> Multivariate Polynome in Sage</h2>
Wir können multivariate Polynomringe wie folgt definieren. $\verb|order="lex"|$ weist SageMath an, die Monome lexikografisch zu ordnen; wenn es weggelassen wird, wird es eine andere Wohlordnung für die Monome wählen (die sogenannte "degree reverse lexicographic order", die in der Vorlesung nicht behandelt wurde).

In [1]:
R.<x1,x2,x3,x4> = PolynomialRing(QQ,order="lex")

$\verb|f.monomials()|$ gibt die Liste der Monome zurück, lexikographisch geordnet.

$\verb|f.coefficients()|$ gibt die Liste der Koeffiziente, in der gleichen Reihenfolge.

In [2]:
g=x1*x4+7*x2*x3+2*x1
print(g.monomials())
print(g.coefficients())

[x1*x4, x1, x2*x3]
[1, 2, 7]


">" vergleicht Monome in Bezug auf die gewählte Monomordnung (und <,<=,>= ähnlich).

In [3]:
x1*x4>x2*x3

True

Verwenden Sie .divides(), um zu testen, ob ein Monom durch ein anderes teilbar ist.

In [4]:
x1.divides(x1*x2)

True

Um die Monome dann zu teilen, verwenden Sie "//" (und nicht "/", dann behandelt Sage es als rationale Funktion anstelle eines Polynoms).

In [5]:
(x1*x2)//x1

x2

<h2> Aufgabe </h2>

Implementieren Sie die Funktion Teilung.

In [6]:
import itertools

def _find_hi_dividing_remainder_term(h, r):
    # Tuples of (i, h_i) where h_i are values from h, i their indices
    for i, hi in enumerate(h):
        # Tuples of (c, x^a), where c * x^a is a term of r, and x^a a monomial
        for c, xa in zip(r.coefficients(), r.monomials()):
            # Leading monomial of h_i
            lm = hi.lm()

            if lm.divides(xa):
                return i, hi, c, xa, True
        
    # None found
    return None, None, None, None, False # Just pretend it's Go

def _check_invariant(q, h, r, f):
    # Check loop invariant for debugging purposes

    # 1) sum(q_i * h_i) + r = f
    assert sum([q[i]*h[i] for i in range(len(h))])+r == f

    # 2) all monomials of r are <= LM(f)
    for rm in r.monomials():
        assert rm <= f.lm()

    # 3) all monomials in product q_i * h_i are <= LM(f)
    for i in range(len(q)):
        for m in (q[i]*h[i]).monomials():
            assert m <= f.lm()


def teilung(f, h):
    # Enforce h_i being monic
    for hi in h:
        if hi.lc() != 1:
            raise ValueError(f"h_i must be monic, but was {hi}")
    
    # q_i = 0
    q = [0]*len(h)        
    r = f
    
    while True:
        i, hi, c, xa, found = _find_hi_dividing_remainder_term(h, r)
        
        if found:
            lt = hi.lt()
            # Update remainder & quotient
            r -= c * (xa // lt) * hi
            q[i] += c * (xa // lt)
            
            # Sufficient to check invariant if we actually performed any work
            if CHECK_INVARIANT:
                _check_invariant(q, h, r, f)
            
        else:
            # No more h_i found such that LM(h_i) divides a monomial of r
            return r, q

Sie können den folgenden Code verwenden, um Ihre Funktion zu testen (wenn Sie möchten, können Sie f und h durch andere Polynome ersetzen). Wenn es ausgeführt wird, ohne dass ein "Assertion Error" ausgelöst wird, bedeutet dies, dass das Ergebnis korrekt ist.

In [7]:
# Check invariants within loop body for testing purposes
CHECK_INVARIANT = True

def test_divide():
    fs = [
        x1^3*x3^2 + x2^2*x4^5*x1^3,
        x1^5*x3^2,
        x1^3*x3^2
    ]

    h=[x1^2+x1*x2^2,x1^2*x3+x4]

    for f in fs:
        print(f"Testing with f = {f}")
        (r,q)=teilung(f,h)
        print(f"\tDivisors: {h}")
        print(f"\tQuotients: {q}")
        print(f"\tRemainder: {r}")
        k=len(h)

        # sum(q_i * h_i) + r = f
        assert(sum([q[i]*h[i] for i in range(k)])+r==f)

        # LM(h_i) does not divide any monomial of r
        for hi in h:
            for mon in r.monomials():
                assert not hi.monomials()[0].divides(mon)

        for i in range(k):
            # q_i * h_i < LM(f)
            assert q[i]==0 or (q[i]*h[i]).monomials()[0] <= f.monomials()[0]
            
test_divide()

Testing with f = x1^3*x2^2*x4^5 + x1^3*x3^2
	Divisors: [x1^2 + x1*x2^2, x1^2*x3 + x4]
	Quotients: [x1*x2^2*x4^5 + x1*x3^2 - x2^4*x4^5 - x2^2*x3^2, 0]
	Remainder: x1*x2^6*x4^5 + x1*x2^4*x3^2
Testing with f = x1^5*x3^2
	Divisors: [x1^2 + x1*x2^2, x1^2*x3 + x4]
	Quotients: [x1^3*x3^2 - x1^2*x2^2*x3^2 + x1*x2^4*x3^2 - x2^6*x3^2, 0]
	Remainder: x1*x2^8*x3^2
Testing with f = x1^3*x3^2
	Divisors: [x1^2 + x1*x2^2, x1^2*x3 + x4]
	Quotients: [x1*x3^2 - x2^2*x3^2, 0]
	Remainder: x1*x2^4*x3^2
